In [1]:
import numpy as np

In [2]:
# Generate synthetic data
np.random.seed(123)
n_samples = 100
x1 = np.random.randn(n_samples)
x2 = np.random.randn(n_samples)
true_beta0 = 1.5
true_beta1 = 2.0
true_beta2 = -1.0
rho_true = 0.8
sigma_true = 0.5
sigma_ar_true = 0.4

In [3]:
# Latent AR(1) process
e = np.zeros(n_samples)
for t in range(1, n_samples):
    e[t] = rho_true * e[t-1] + np.random.normal(0, sigma_ar_true)

In [4]:
# Linear model with AR(1) errors
y = true_beta0 + true_beta1 * x1 + true_beta2 * x2 + e + np.random.normal(0, sigma_true, size=n_samples)


In [5]:
x1.shape, x2.shape

((100,), (100,))

In [6]:
# Define the log-posterior function
def log_posterior(beta0, beta1, beta2, rho, sigma, sigma_ar):
    # Log-prior for beta coefficients
    log_prior = (-0.5 * (beta0**2 + beta1**2 + beta2**2) / 10**2)  # Normal prior
    
    # Log-prior for AR(1) coefficient
    if not (-1 < rho < 1):
        return -np.inf  # Reject if out of bounds
    log_prior += 0  # Uniform prior for rho

    # Log-prior for sigma and sigma_ar
    if sigma <= 0 or sigma_ar <= 0:
        return -np.inf  # Reject if not positive
    log_prior += np.log(1 / (sigma * 5)) # Half-normal prior for sigma
    log_prior += np.log(1 / (sigma_ar * 5))  # Half-normal prior for sigma_ar
    
    # Calculate the log-likelihood
    mu = beta0 + beta1 * x1 + beta2 * x2
    likelihood = 0
    for t in range(n_samples):
        # Latent AR(1) error
        if t == 0:
            e_t = 0  # Start with zero for the first value
        else:
            e_t = rho * e[t-1] + np.random.normal(0, sigma_ar)
        mu_t = mu[t] + e_t
        likelihood += -0.5 * np.log(2 * np.pi * sigma**2) - (y[t] - mu_t)**2 / (2 * sigma**2)

    return log_prior + likelihood

In [7]:
# NUTS sampling
def nuts(initial_params, no_of_samples, step_size=0.1):
    samples = []
    current_params = np.array(initial_params)
    
    for _ in range(no_of_samples):
        # Sample the new parameters using NUTS
        new_params = current_params + np.random.normal(0, step_size, size=len(current_params))
        current_log_posterior = log_posterior(*current_params)
        new_log_posterior = log_posterior(*new_params)

        # Accept or reject the new parameters based on the acceptance criterion
        if np.random.uniform(0, 1) < np.exp(new_log_posterior - current_log_posterior):
            current_params = new_params  # Accept the new parameters
        
        samples.append(current_params)

    return np.array(samples)


In [8]:
%%time
# Initial parameters for sampling
initial_params = [0.0, 0.0, 0.0, 0.0, 1.0, 1.0]  # beta0, beta1, beta2, rho, sigma, sigma_ar
no_of_samples = 20000

CPU times: user 3 µs, sys: 3 µs, total: 6 µs
Wall time: 11.4 µs


In [9]:
# Run the NUTS sampler
samples = nuts(initial_params, no_of_samples)

In [10]:
# Print the summary of the samples
print("Sampled Coefficients:")
print("Beta0:", np.mean(samples[:, 0]))
print("Beta1:", np.mean(samples[:, 1]))
print("Beta2:", np.mean(samples[:, 2]))
print("Rho:", np.mean(samples[:, 3]))
print("Sigma:", np.mean(samples[:, 4]))
print("Sigma AR:", np.mean(samples[:, 5]))


Sampled Coefficients:
Beta0: 1.390540495067725
Beta1: 1.9644589141651627
Beta2: -0.9367205899211698
Rho: 0.8579927299591774
Sigma: 0.6776163470764631
Sigma AR: 0.02006163111147225
